In [88]:
#načítanie knihy 
kniha1 = open('./Rivers of Babylon.txt', 'r', encoding="utf8")
kniha1 = kniha1.read()


In [89]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

slova = word_tokenize(kniha1.lower())
slova [:30]


[nltk_data] Downloading package punkt to /home/lubolacko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['ráno',
 'sa',
 'kotolník',
 'zobúdza',
 's',
 'takou',
 'nenávisťou',
 'v',
 'duši',
 ',',
 'že',
 'mu',
 'ani',
 'jesť',
 'nechutí',
 '.',
 'vylihuje',
 'na',
 'drevenej',
 'lavici',
 ',',
 'škrabe',
 'si',
 'svrbiacu',
 'kožu',
 ',',
 'napnutú',
 'na',
 'vysadených',
 'rebrách']

In [90]:
import string
punct = string.punctuation
punct 


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [91]:
import string
zoznam_slov = []
neabecedne_znaky = string.punctuation
neabecedne_znaky

for slovo in slova:
        for char in slovo:
            if char in neabecedne_znaky:
                slovo = slovo.replace(char, "")
        if slovo not in neabecedne_znaky:
            zoznam_slov.append(slovo)
print("Počet slov:", len(zoznam_slov), "unikátnych", len(set(zoznam_slov)))
zoznam_slov [:30]


Počet slov: 90374 unikátnych 19679


['ráno',
 'sa',
 'kotolník',
 'zobúdza',
 's',
 'takou',
 'nenávisťou',
 'v',
 'duši',
 'že',
 'mu',
 'ani',
 'jesť',
 'nechutí',
 'vylihuje',
 'na',
 'drevenej',
 'lavici',
 'škrabe',
 'si',
 'svrbiacu',
 'kožu',
 'napnutú',
 'na',
 'vysadených',
 'rebrách',
 'a',
 'tupo',
 'sa',
 'obzerá']

In [92]:
slovnik= set(zoznam_slov) #unikátne slová
print("Slovná zásoba na trénovanie má", len(zoznam_slov),"slov, unikátnych slov je", len(slovnik))
indexy_slov = {slovo: i for i, slovo in enumerate(slovnik)}
zoznam_indexov_slov = [indexy_slov[slovo] for slovo in zoznam_slov]
zoznam_indexov_slov[:10]


Slovná zásoba na trénovanie má 90374 slov, unikátnych slov je 19679


[1008, 878, 2397, 9446, 8970, 6174, 5474, 13103, 11210, 8016]

In [93]:
davka=5
pocet_slov = len(zoznam_slov) #pocet slov knihy
trenovacia_mnozina = [([zoznam_indexov_slov[i], zoznam_indexov_slov[i+1], zoznam_indexov_slov[i+2], zoznam_indexov_slov[i+3], zoznam_indexov_slov[i+4]], zoznam_indexov_slov[i+5]) for i in range(pocet_slov-davka) ]


In [94]:
trenovacia_mnozina[:10]

[([1008, 878, 2397, 9446, 8970], 6174),
 ([878, 2397, 9446, 8970, 6174], 5474),
 ([2397, 9446, 8970, 6174, 5474], 13103),
 ([9446, 8970, 6174, 5474, 13103], 11210),
 ([8970, 6174, 5474, 13103, 11210], 8016),
 ([6174, 5474, 13103, 11210, 8016], 14215),
 ([5474, 13103, 11210, 8016, 14215], 13303),
 ([13103, 11210, 8016, 14215, 13303], 10100),
 ([11210, 8016, 14215, 13303, 10100], 4565),
 ([8016, 14215, 13303, 10100, 4565], 12490)]

In [95]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

embedding_dim = 5  #veľkosť embedding vektorov
# === Trieda definujúca model neurónovej siete ===
# Embedding (vložená vrstva) je skrytá vrstva NS ktorá
# mapuje vstupné informácie z vysoko do nízko rozmerného priestoru

class Model_NS(nn.Module):
  def __init__(self, pocet_slov, embedding_dim, davka):
    super(Model_NS, self).__init__()
    self.embeddings = nn.Embedding(pocet_slov, embedding_dim)
    self.linear1 = nn.Linear(davka * embedding_dim,128)  # lineárne transformácie ako vrstvy NS
    self.linear2 = nn.Linear(128, 512)
    self.linear3 = nn.Linear(512, pocet_slov)

   # pri predpovedaní informácie prúdia „dopredu“
   # zo vstupu cez skryté vrstvy na výstup
   # výstupom sú logaritmy pravdepodobností
  def forward(self, inputs):  # pri predpovedaní
    embs = self.embeddings(inputs).view((1,-1))
    out = F.relu(self.linear1(embs)) #lineárna aktivačná funkcia
    out = F.relu(self.linear2(out))
    out = self.linear3(out)
    log_pravdepodobnosti = F.log_softmax(out, dim=1) #konvertuje číselné hodnoty na pravdepodobnosti
    return log_pravdepodobnosti



In [96]:
# model a výpis parametrov modelu
model = Model_NS(pocet_slov, embedding_dim, davka)
print(model.embeddings,"\n")
model 


Embedding(90374, 5) 



Model_NS(
  (embeddings): Embedding(90374, 5)
  (linear1): Linear(in_features=25, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=90374, bias=True)
)

In [97]:
# === trénovanie modelu CPU ===
device = torch.device("cpu")

In [158]:
import psutil
import psutil
print ("cpu fyzické jadrá: ",psutil.cpu_count(logical=False))
print ("cpu:logické jadrá ",psutil.cpu_count())
print (psutil.cpu_freq())
print (psutil.cpu_percent())
temps = psutil.sensors_temperatures()
print (temps.items())
print("-----")

cpu_temp = psutil.sensors_temperatures()["k10temp"][0]
print(cpu_temp.current)
print("-----")
   
for name, entries in temps.items():
        print(name)
        for entry in entries:
 
            line = "    %-20s %s °C " % (
                entry.label or name,
                entry.current,)
                
            print(line)
            print()

cpu fyzické jadrá:  16
cpu:logické jadrá  32
scpufreq(current=2.3742187500000007, min=400.0, max=5881.0)
1.1
dict_items([('nvme', [shwtemp(label='Composite', current=34.85, high=81.85, critical=85.85)]), ('k10temp', [shwtemp(label='Tctl', current=35.625, high=None, critical=None), shwtemp(label='Tccd1', current=40.25, high=None, critical=None), shwtemp(label='Tccd2', current=28.875, high=None, critical=None)]), ('amdgpu', [shwtemp(label='edge', current=32.0, high=None, critical=None)])])
-----
35.625
-----
nvme
    Composite            34.85 °C 

k10temp
    Tctl                 35.625 °C 

    Tccd1                40.25 °C 

    Tccd2                28.875 °C 

amdgpu
    edge                 32.0 °C 



In [ ]:
# === trénovanie modelu GPU ===
from torch.cuda import is_available

# Pokiaľ možno na GPU
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [168]:
from datetime import timedelta

priebezne_chyby = []  # chyby po každých 1000 krokoch trénovania 

#red = '\033[91m'
#green = '\033[92m'
#yellow = '\033[93m'
#blue = '\033[94m'
#pink = '\033[95m'
#teal = '\033[96m'
#grey = '\033[97m'

psutil.sensors_temperatures() 



user = 'lubolacko' 


def format_td(seconds, digits=3):
    isec, fsec = divmod(round(seconds*10**digits), 10**digits)
    return f'{timedelta(seconds=isec)}.{fsec:0{digits}.0f}'


def trenovanie(model, trenovacie_data, n_iteracie, word_to_index):
  model.to(device)
  for i in range(n_iteracie):
    model.train()
    krok = 0
    perioda_vypisu = 100
    akumulovana_chyba = 0  # na výpočet priemernej chyby po každých 1000 krokoch 
    
    for feature, target in trenovacie_data:
      feature_tensor = torch.tensor([feature], dtype=torch.long)
      feature_tensor = feature_tensor.to(device)
      target_tensor = torch.tensor([target], dtype=torch.long)
      target_tensor = target_tensor.to(device)
      model.zero_grad()  #vynulovanie gradientov
      log_probs = model(feature_tensor)
      loss = criterion(log_probs, target_tensor)
      loss.backward()    #Spätný prechod pri aktualizácii parametrov
      optimizer.step()
      akumulovana_chyba += loss.item()
      krok+=1

      if krok % perioda_vypisu == 0:  #každých 1000 krokov
        model.eval()
        priemerna_chyba = akumulovana_chyba/perioda_vypisu
        priebezne_chyby.append(priemerna_chyba) #pridám do zoznamu
        sec = (time.time()-start_time)
        cpu = psutil.cpu_percent()
        cpu_temp = psutil.sensors_temperatures()["k10temp"][0]
        #print(cpu_temp.current)
        # print ("cpu:freq ",psutil.cpu_freq())
        # print(psutil.cpu_percent(percpu=True))
        print(i+1,"z",n_iteracie, "iterácií", krok,"krok z",pocet_slov,
               "Chyba: {:.3f}".format(priemerna_chyba), 
               "CPU: {:.2f}%".format(cpu),  
               "{:.2f}°C".format(cpu_temp.current),
               " čas {} ".format(format_td(sec, digits=2)))
        
        akumulovana_chyba=0
    model.train()
  return model


In [169]:
model = Model_NS(pocet_slov, embedding_dim, davka)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

n_iteracie = 2

start_time = time.time()

model = trenovanie(model, trenovacia_mnozina, n_iteracie, indexy_slov)
sec = (time.time()-start_time)
mins = sec/60

print("Trénovanie modelu trvalo {} minút".format(mins), 
      "--- {} ".format(format_td(sec, digits=2)))  

# Doplnenie modelu
model.average_loss = priebezne_chyby
model.optimizer_state = optimizer.state_dict()
model.word_to_index = indexy_slov,
model.index_to_word = {i: word for i, word in enumerate(slovnik)}


1 z 2 iterácií 100 krok z 90374 Chyba: 11.401 CPU: 4.60% 56.50°C  čas 0:00:08.89 
1 z 2 iterácií 200 krok z 90374 Chyba: 11.343 CPU: 50.80% 58.38°C  čas 0:00:17.82 
1 z 2 iterácií 300 krok z 90374 Chyba: 11.288 CPU: 50.50% 59.50°C  čas 0:00:26.75 


KeyboardInterrupt: 

In [15]:
import pandas as pd
import random


def predpoved(model, zaciatok_poviedky ,dlzka_poviedky ,top_slova):
    # top_slova - slovo sa vyberie náhodne z top_k počtu

    # Prvých 5 slov do poviedky
    vstupy = (zaciatok_poviedky.lower()).split(" ") # Prvých 5 slov
    for i in vstupy:
        poviedka.append(i)
    for i in range(dlzka_poviedky):
        # slová na čísla (indexy)
        vstupy_ix = torch.tensor([indexy_slov[word] for word in vstupy], dtype=torch.long)
        vstupy_ix = vstupy_ix.to(device)
        with torch.no_grad(): # nevoláme Tensor.backward() nemusíme počítať gradienty
            output = model.double().forward(vstupy_ix)
        ps = torch.exp(output)
        top_kombinacia_indexy = ps.topk(top_slova, sorted=True)
        #top_n najpravdepodobnejších slov
        topk_ps = top_kombinacia_indexy[0][0]
        top_ntica = top_kombinacia_indexy[1][0]
        top_ntica = [model.index_to_word[int(i)] for i in top_ntica]
        # náhodný výber slova z top_n
        next_word = random.choice(top_ntica)
        vstupy = vstupy[1:]  # pridanie slova do vstupov a posunutie
        vstupy.append(next_word)
        poviedka.append(next_word)  # pridanie slova do poviedky
    return poviedka


In [18]:
zaciatok_poviedky = input('Zadaj prvých {} slov poviedky:\n'.format(davka))

top_slova = 3 # Slovo sa vyberie náhodne z top_k počtu
dlzka_poviedky = 100
poviedka = []

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

# osetrenie vstupu, treba zadať 5 slov, ktoré model pozná
try:
    predikcia = predpoved(model, zaciatok_poviedky, dlzka_poviedky, top_slova)
except KeyError as error:
    print('niektorému zo slov zatiaľ nerozumiem\n zadaj slová z kníh na ktorých som sa učil nie ', error)
    if poviedka[0] != "":
        poviedka = poviedka[len(zaciatok_poviedky):]
    zaciatok_poviedky = input('zadaj slová ktorým rozumiem\n')
    predikcia = predpoved(model, zaciatok_poviedky, dlzka_poviedky, top_slova)
except KeyError and RuntimeError:
    if poviedka[0] != "":
        poviedka = poviedka[len(zaciatok_poviedky):]
    zaciatok_poviedky = input('Zadal si {} slov namiesto {}!\n\nZadaj presný počet slov\n'.format(len(zaciatok_poviedky.split(" ")), 5))
    predikcia = predpoved(model, zaciatok_poviedky, dlzka_poviedky, top_slova)

print('\n=== Poviedka ===')
print(" ".join(poviedka))


Zadaj prvých 5 slov poviedky:
obloha čoraz častejšie dostáva farbu

=== Poviedka ===
obloha čoraz častejšie dostáva farbu sa zdržuje rácz a o za pyšného po snežných pláňach stále dohora stále na sever psy ktoré z mesta po zdá si napokon pred hotelom no čo sa fiškál nasťahujú som dve ako všetkým sa dážď rozhodne už dávno po meste si reštauráciou sa už vyriešila vlasti na všetko deň svoju prídu sa fiškál unaví samozrejme stranu a na fiškála nikde hypotéku sa budú na mesto to on rácz vymyslel potrebovať nepozerá na jeseň ale no čo čo povedať rácz a nebola kohosi spokojne hotelom ambassador kravatu sa nik do si a pevne švéd a po celom meste si fiškáľa
